## Tensorflow Implementation

In [1]:
import tensorflow as tf
import numpy as np
from sklearn.preprocessing import StandardScaler

In [23]:
(train_features, train_target), (test_features, test_target) = tf.keras.datasets.boston_housing.load_data()

In [24]:
tf.keras.backend.set_floatx('float64')
train_target = tf.reshape(train_target, (-1, 1))
test_target = tf.reshape(test_target, (-1, 1))

In [25]:
scaler = StandardScaler()
train_features = scaler.fit_transform(train_features)
test_features = scaler.transform(test_features)

In [26]:
def load_data(data, batch_size):
    dataset = tf.data.Dataset.from_tensor_slices(data)
    dataset = dataset.shuffle(buffer_size = 100)
    dataset = dataset.batch(batch_size)
    return dataset

batch_size = 64
data_iter = load_data((train_features, train_target), batch_size)
# next(iter(data_iter))

In [27]:
def log_rmse(y_true, y_pred):
    # To further stabilize the value when the logarithm is taken, set the
    # value less than 1 as 1
    clipped_preds = tf.clip_by_value(y_pred, 1, float('inf'))
    return tf.sqrt(tf.reduce_mean(loss(
        tf.math.log(y_true), tf.math.log(clipped_preds))))

In [34]:
lin_reg = tf.keras.models.Sequential()
lin_reg.add(tf.keras.layers.Dense(10, activation = 'relu'))
lin_reg.add(tf.keras.layers.Dense(5, activation = 'relu'))
lin_reg.add(tf.keras.layers.Dense(3, activation = 'relu'))
lin_reg.add(tf.keras.layers.Dense(1))

In [35]:
optimizer = tf.keras.optimizers.Adam(learning_rate= 0.1)
loss = tf.keras.losses.MeanSquaredError()

In [36]:
num_epoch = 10
batch_size = 32
data_iter = load_data((train_features, train_target), batch_size)
for epoch in range(num_epoch):
    for X, y in data_iter:
        with tf.GradientTape() as tape:
            yhat = lin_reg(X)
            mse_loss = loss(yhat, y)
            
        # Gradient
        params = lin_reg.trainable_variables
        grads = tape.gradient(mse_loss, params)

        # Update Gradient
        optimizer.apply_gradients(zip(grads, params))
    
    training_loss = loss(train_target, lin_reg(train_features))
    print(f"epoch : {epoch}, training_loss : {training_loss}")

epoch : 0, training_loss : 62.75815963745117
epoch : 1, training_loss : 24.664142608642578
epoch : 2, training_loss : 28.78792953491211
epoch : 3, training_loss : 17.72987937927246
epoch : 4, training_loss : 11.17648696899414
epoch : 5, training_loss : 12.671730041503906
epoch : 6, training_loss : 9.46349811553955
epoch : 7, training_loss : 9.49169635772705
epoch : 8, training_loss : 8.649161338806152
epoch : 9, training_loss : 8.545610427856445
